In [ ]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# load the saved H5 model
loaded_model = load_model("/models/txt_em_conf_model/nlp_model.h5")

# Load tokenizer
with open('/models/txt_em_conf_model/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Load encoder
with open('/models/txt_em_conf_model/encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
def predict(text):
    sequences = tokenizer.texts_to_sequences([text])
    x_new = pad_sequences(sequences, maxlen=50)
    predictions = loaded_model.predict([x_new, x_new])

    # Get labels and probabilities
    labels = list(encoder.classes_)
    probs = list(predictions[0])

    results = []

    for label, conf in zip(labels, probs):
        results.append({'label': label, 'confidence': conf})

    return results

# Text Emotion

In [ ]:
input_sentence = '''we run up the mountain yesterday the night sky was full of stars I've never seen so many stars in the sky before it was such a heavenly view forever remeber it in the bottom of my heart'''

speech_results = predict(input_sentence)

1/1 [==============================] - 0s 21ms/step


In [ ]:
print("Predicted Emotions and Confidences:", f"{speech_results[0]['label']}, {speech_results[0]['confidence']}\n")

speech_results

Predicted Emotions and Confidences: Anger, 0.3505152463912964



[{'label': 'Anger', 'confidence': 0.35051525},
 {'label': 'Calmness', 'confidence': 0.22287634},
 {'label': 'Happiness', 'confidence': 0.23102723},
 {'label': 'Sadness', 'confidence': 0.19558118}]

Lyrics Emotions

In [ ]:
input_lyrics = '''
"Hey, Nate, how's life?"
I don't know, it's alright
I've been dealin' with some things like every human being
And really didn't sleep much last night
"I'm sorry," that's fine
I just think I need a little me time
I just think I need a little free time
Little break from the shows and the bus rides, yeah
Last year I had a breakdown
Thoughts tellin' me I'm lost gettin' too loud
Had to see a therapist, then I found out
Somethin' funny's goin' on up in my house
Yeah, I started thinkin' maybe I should move out
You know, pack my cart, take a new route

Clean up my yard, get the noose out
Hang up my heart, let it air out
I've been searchin'
"What does that mean, Nate?" I've been learning
Grabbin' my keepsakes, leavin' my burdens
Well, I brought a few with me, I'm not perfect
Lookin' at the view, like this concerns me
Pickin' up the cues, right? I'm quite nervous
Hate it when I lose sight, life gets blurry
And things might hurt me
It's prolly gonna be a long journey, but hey
It's worth it, though
Cold world out there, kids, grab your coats
It's been a minute, I know, now I'm back to roam
Lookin' for the antidote to crack the code
Pretty vivid; I admit it, I'm in classic mode
Don't need pity given to me, but I can't condone
Talkin' down to me, I'ma have to crack your nose for crackin' jokes

I'm lookin' for the map to hope, you seen it?
Been makin' a whole lot of changes
Wrote a song about that, you should play it
I get scared when I walk on these stages
I look at the crowd and see so many faces, yeah
That's when I start to get anxious
That's when my thoughts can be dangerous
That's when I put on my makeup and drown in self-hatred
Forget what I'm saying, and—
Where'd the beat go?
Oh, ain't that somethin'?
Drums came in, you ain't see that comin'
Hands on my head, can't tell me nothin'
Got a taste of the fame, had to pump my stomach
Throw it back up like I don't want it
Wipe my face, clean off my vomit
OCD, tryna push my buttons

I said don't touch it, now y'all done it
I can be critical, never typical
Intricate with every syllable, I'm a criminal
Intimate, but never political, pretty visual
Even if you hate it, I'll make it feel like you're in it, though
You call me what you wanna, but never call me forgettable
Leave you deep in thought, I could never swim in the kiddie pool
Way that I been thinkin' is cinematic, it's beautiful
Man, I don't know if I'm makin' movies or music videos (Videos, videos)
Yeah, the sales can rise
Doesn't mean much though when your health declines
See, we've all got somethin' that we trapped inside

That we try to suffocate, you know, hopin' it dies
Try to hold it underwater but it always survives
Then it comes up out of nowhere like an evil surprise
Then it hovers over you to tell you millions of lies
You don't relate to that? Must not be as crazy as I am
The point I'm makin' is the mind is a powerful place
And what you feed it can affect you in a powerful way
It's pretty cool, right? Yeah, but it's not always safe
Just hang with me, this'll only take a moment, okay?
Just think about it for a second, if you look at your face

Every day when you get up and think you'll never be great
You'll never be great—not because you're not, but the hate
Will always find a way to cut you up and murder your faith (Woo)
I've been developin', take a look at the benefits
Nothin' to meddle with, I can never be delicate
Am I even relevant? That depends how you measure it
Take a measurement, then bag it up and give me the evidence
Pretty evident; dependable can never be tentative
I'm a gentleman, depending on if I think you're genuine
Pretty elegant, but not afraid to tell you to get a grip
Proper etiquette, I keep it to myself when I celebrate, ah

It's that time again
Better grab your balloons and invite your friends
Seatbelts back on, yeah, strap 'em in
Look at me, everybody, I'm smilin' big
On a road right now that I can't predict
Tell me "Tone that down," but I can't resist
Y'all know that sound, better raise your fist
The search begins, I'm back, so enjoy the trip, huh
'''

lyrics_results = predict(input_lyrics)

1/1 [==============================] - 0s 22ms/step


In [ ]:
print("Predicted Emotions and Confidences:", f"{lyrics_results[0]['label']}, {lyrics_results[0]['confidence']}\n")

lyrics_results

Predicted Emotions and Confidences: Anger, 0.00476447818800807



[{'label': 'Anger', 'confidence': 0.004764478},
 {'label': 'Calmness', 'confidence': 0.01103484},
 {'label': 'Happiness', 'confidence': 0.96298},
 {'label': 'Sadness', 'confidence': 0.021220798}]

# Find Similarity

In [ ]:
def calculate_similarity_conf(output1, output2):
    total_difference = 0.0

    num_labels = len(output1)

    for i in range(num_labels):
        label1 = output1[i]['label']
        confidence1 = output1[i]['confidence']

        label2 = output2[i]['label']
        confidence2 = output2[i]['confidence']

        if label1 == label2:
            total_difference += abs(confidence1 - confidence2)

    similarity = (1 - total_difference / num_labels) * 100
    return similarity

In [ ]:
similarity_percent = calculate_similarity_conf(speech_results, lyrics_results)
print("Overall similarity: {:.2f}%".format(similarity_percent))

Overall similarity: 63.40%


Matching Labels:

We compare each label from output1 with the corresponding label from output2:

Anger -> Anger

Happiness -> Happiness

Calmness -> Calmness

Sadness -> Sadness

Calculating Differences in Confidence Scores:

For each matched label, we calculate the absolute difference between the confidence scores:

For Anger: |0.35051525 - 0.004764478| ≈ 0.345750772

For Happiness: |0.23102723 - 0.96298| ≈ 0.73195277

For Calmness: |0.22287634 - 0.01103484| ≈ 0.2118415

For Sadness: |0.19558118 - 0.021220798| ≈ 0.174360382

Total Similarity:

We sum up these absolute differences:
0.345750772 + 0.73195277 + 0.2118415 + 0.174360382 ≈ 1.463905424

Average Similarity:

We divide the total similarity by the number of matched labels (which is 4 in this case):

Average Similarity = (1 - Total Similarity / Number of matched labels) * 100

Average Similarity = (1 - 1.463905424 / 4) * 100 ≈ (1 - 0.365976356) * 100 ≈ 0.634023644 * 100 ≈ 63.40%

Overall Similarity: 63.40%

The overall similarity is approximately 63.40%.